In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
!ls -lh ../data

total 92544
-rw-r--r--  1 felix  staff    51K Feb 18 12:03 ai_simulation_20200218-110334.csv
-rw-r--r--  1 felix  staff    51K Feb 18 12:11 ai_simulation_20200218-111125.csv
-rw-r--r--  1 felix  staff    51K Feb 18 12:13 ai_simulation_20200218-111344.csv
-rw-r--r--  1 felix  staff    50K Feb 18 13:52 ai_simulation_20200218-125231.csv
-rw-r--r--  1 felix  staff    50K Feb 18 14:34 ai_simulation_20200218-133401.csv
-rw-r--r--  1 felix  staff    51K Feb 18 14:48 ai_simulation_20200218-134808.csv
-rw-r--r--  1 felix  staff    51K Feb 18 15:04 ai_simulation_20200218-140412.csv
-rw-r--r--  1 felix  staff    52K Feb 18 15:06 ai_simulation_20200218-140649.csv
-rw-r--r--  1 felix  staff    51K Feb 18 16:03 ai_simulation_20200218-150352.csv
-rw-r--r--  1 felix  staff    51K Feb 18 16:06 ai_simulation_20200218-150654.csv
-rw-r--r--  1 felix  staff   516K Feb 18 16:25 ai_simulation_20200218-152534.csv
-rw-r--r--  1 felix  staff   514K Feb 18 16:27 ai_simulation_20200218-152711.csv
-rw-r--r--  1 fe

In [3]:
DATA_PATH = "../data/modern_march_20200226-145902.csv"

In [4]:
# load data
data = pd.read_csv(DATA_PATH)
data.shape

(818100, 10)

In [5]:
data.head()

,config,run,step,belief_dims,num_humans,lr_hum_org,lr_org_hum,lr_hum_dat,code_kl,human_kl
0,1,1,0,30,50,0.10,0.10,0.05,0.00,0.36
1,1,1,1,30,50,0.10,0.10,0.05,0.40,0.39
2,1,1,2,30,50,0.10,0.10,0.05,0.53,0.41
3,1,1,3,30,50,0.10,0.10,0.05,0.53,0.44
4,1,1,4,30,50,0.10,0.10,0.05,0.60,0.47


In [6]:
# round in order to enable secure indexing
data = data.round(2)

In [7]:
# reindex
configs = pd.unique(data.config)
runs = pd.unique(data.run)
steps = pd.unique(data.step)
index = pd.MultiIndex.from_product([configs, runs, steps], names=['config', 'run', 'step'])
data.index = index
data = data.drop(columns=['config', 'run', 'step'])
data.head()

belief_dims  num_humans  lr_hum_org  lr_org_hum  lr_hum_dat  \
config run step                                                                
1      1   0              30          50        0.10        0.10        0.05   
           1              30          50        0.10        0.10        0.05   
           2              30          50        0.10        0.10        0.05   
           3              30          50        0.10        0.10        0.05   
           4              30          50        0.10        0.10        0.05   

                 code_kl  human_kl  
config run step                     
1      1   0        0.00      0.36  
           1        0.40      0.39  
           2        0.53      0.41  
           3        0.53      0.44  
           4        0.60      0.47

In [8]:
# aggregate over runs with same configuration
time_data = data.groupby(level=[0, 2]).agg('mean')
time_data.shape

(8181, 7)

In [9]:
time_data.head()

belief_dims  num_humans  lr_hum_org  lr_org_hum  lr_hum_dat  \
config step                                                                
1      0              30          50        0.10        0.10        0.05   
       1              30          50        0.10        0.10        0.05   
       2              30          50        0.10        0.10        0.05   
       3              30          50        0.10        0.10        0.05   
       4              30          50        0.10        0.10        0.05   

             code_kl  human_kl  
config step                     
1      0        0.00      0.33  
       1        0.18      0.37  
       2        0.36      0.39  
       3        0.51      0.42  
       4        0.58      0.46

In [10]:
# aggregate over time steps
aggs = {
    "belief_dims": "mean",
    "num_humans": "last",
    "lr_hum_org": "mean",
    "lr_org_hum": "mean",
    "lr_hum_dat": "mean",
    "code_kl": ["max", "last"],
    "human_kl": ["max", "last"],
}
agg_data = time_data.groupby(level=0).agg(aggs)
agg_data.shape

(81, 9)

In [11]:
# flatten column identifiers
agg_data.columns = ['_'.join(col).strip() for col in agg_data.columns.values]
agg_data.head()

,belief_dims_mean,num_humans_last,lr_hum_org_mean,lr_org_hum_mean,lr_hum_dat_mean,code_kl_max,code_kl_last,human_kl_max,human_kl_last
config,,,,,,,,,
1,30,50,0.10,0.10,0.05,1.00,1.00,1.00,1.00
2,30,50,0.10,0.20,0.05,1.00,1.00,1.00,1.00
3,30,50,0.10,0.30,0.05,1.00,1.00,1.00,1.00
4,30,50,0.10,0.40,0.05,1.00,1.00,1.00,1.00
5,30,50,0.10,0.50,0.05,1.00,1.00,1.00,1.00


In [12]:
# rename columns for convenience
cols = {
    "belief_dims_mean": "belief_dims",
    "num_humans_last": "num_humans",
    "lr_hum_org_mean": "lr_hum_org",
    "lr_org_hum_mean": "lr_org_hum",
    "lr_hum_dat_mean": "lr_hum_dat",
}
agg_data.rename(columns=cols, inplace=True)
agg_data.head()

,belief_dims,num_humans,lr_hum_org,lr_org_hum,lr_hum_dat,code_kl_max,code_kl_last,human_kl_max,human_kl_last
config,,,,,,,,,
1,30,50,0.10,0.10,0.05,1.00,1.00,1.00,1.00
2,30,50,0.10,0.20,0.05,1.00,1.00,1.00,1.00
3,30,50,0.10,0.30,0.05,1.00,1.00,1.00,1.00
4,30,50,0.10,0.40,0.05,1.00,1.00,1.00,1.00
5,30,50,0.10,0.50,0.05,1.00,1.00,1.00,1.00
